# Data Cleaning

Esse notebook tem por objetivo fazer a limpeza dos dados fornecidos e torná-los próprios para análise.

In [1]:
import pandas as pd
import os
from pprint import pprint

from config import DIR_DADOS_GERADOS

### função para carregar os dados em csv

In [2]:
def load_base_csv(fname:str, **read_kwargs)->pd.DataFrame:
    """Carrega o arquivo CSV da base orginal e retorna um DataFrame."""

    file = os.path.join('data', 'bases', fname)
    if not os.path.exists(file):
        raise FileNotFoundError(f"Arquivo {file} não encontrado.")
    return pd.read_csv(file, **read_kwargs)

### Dicionário de variáveis

Foi fornecido um dicionário de variáveis

In [3]:
dicionario = load_base_csv('dicionário_bases.csv', sep=';', skiprows=1)

In [4]:
dicionario

,Nome,Descrição,Respostas
0,cod_familiar,código familiar,[código]
1,localidade,Localidade do endereço,[nome da localidade]
2,faixa_renda,Faixas de renda familiar mensal per capita,2 = R$ 0 a R$ 600 reais; 3 = R$ 601 a R$ 1000;...
3,data_nascimento,Data de nascimento da pessoa,[Data]
4,sexo,Sexo,1 = Masculino; 2 = Feminino
5,parentesco,Relação de parentesco com o responsável familiar,1 = Pessoa Responsavel Familiar; 2 = Conjuge/c...
6,escolaridade,Grau de escolaridade,1 = Sem instrucao; 2 = Fundamental incompleto;...
7,referencia_dados,Data de referência da extração da base de dados,"[Data, somente com mês e ano, considere o dia ..."


In [5]:
dicionario.shape

(8, 3)

### Bases CSV fornecidas

Foram fornecidas duas bases csv:
* test_fam_v2.csv -- que contém dados sobre as famílias
* test_pes_v2.csv -- que contém dados sobre as pessoas que pertencem às famílias

In [6]:
fam = load_base_csv('test_fam_v2.csv', sep=';')

In [7]:
fam.head()

,cod_familiar,localidade,faixa_renda,referencia_dados
0,X*HXV;+W,VILA DELLA PIAZZA,2,2024-01
1,X@;WXW?H,PARQUE ELOY CHAVES,3,2024-01
2,**+@?*WXV,JARDIM SANTA GERTRUDES,2,2024-01
3,*YX*;Y;XY,JARDIM SAO CAMILO,3,2024-01
4,;**?;;++V,VILA MARINGA,2,2024-01


In [8]:
fam.shape

(202385, 4)

In [9]:
pess = load_base_csv('test_pes_v2.csv', sep=';')

In [10]:
pess.head()

,cod_familiar,data_nascimento,sexo,parentesco,escolaridade,referencia_dados
0,X*HXV;+W,04/05/1967,2,1.0,1.0,2024-01
1,X*HXV;+W,29/05/2006,2,5.0,2.0,2024-01
2,X@;WXW?H,24/08/2006,2,3.0,3.0,2024-01
3,X@;WXW?H,16/03/1984,2,1.0,4.0,2024-01
4,**+@?*WXV,11/09/1992,2,3.0,5.0,2024-01


In [11]:
pess.shape

(497291, 6)

### Merge Bases

Como os dados não são muito grandes, podemos dar join nas bases de dados para facilitar a análise.

Vamos injetar os dados sobre as famílias na base de pessoas.

A base de famílias é importante pois contém a faixa de renda, que define a eligibilidade. Além do bairro que permitirá fazer os mapas coropléticos posteriormente.

Podemos reconstruir a unidade de análise "família" posteriormente por meio de groupby. Isso será importante por exemplo para calcular a distribuição de gênero e escolaridade na família.

Precisa ser um left join com pessoas à esquerda: ou seja, vamos inserir os dados da família sobre a pessoa.

In [12]:
cods_familias = set(fam['cod_familiar'].unique())
cods_familias_pessoas = set(pess['cod_familiar'].unique())

In [13]:
if not (cods_familias - cods_familias_pessoas):
    print("Todas as famílias tem membros na base de pessoas.")

Todas as famílias tem membros na base de pessoas.


In [14]:
if not (cods_familias_pessoas - cods_familias):
    print("As famílias de todas as pessoas estão identificadas na base de famílias.")

As famílias de todas as pessoas estão identificadas na base de famílias.


In [15]:
df = pd.merge(pess, fam, how='left')

In [16]:
df.head()

,cod_familiar,data_nascimento,sexo,parentesco,escolaridade,referencia_dados,localidade,faixa_renda
0,X*HXV;+W,04/05/1967,2,1.0,1.0,2024-01,VILA DELLA PIAZZA,2
1,X*HXV;+W,29/05/2006,2,5.0,2.0,2024-01,VILA DELLA PIAZZA,2
2,X@;WXW?H,24/08/2006,2,3.0,3.0,2024-01,PARQUE ELOY CHAVES,3
3,X@;WXW?H,16/03/1984,2,1.0,4.0,2024-01,PARQUE ELOY CHAVES,3
4,**+@?*WXV,11/09/1992,2,3.0,5.0,2024-01,JARDIM SANTA GERTRUDES,2


In [17]:
len(df) == len(pess)

True

In [18]:
del pess
del fam
del cods_familias_pessoas
del cods_familias

### Reconstruir as variávies categóricas

Agora podemos reconstruir as variáveis categóricas (que foram fornecidas como ids).

O dicionário de variáveis tem os dados das categorias, que foram transcritos nos mapeamentos abaixo.

In [19]:
cols_categoricas = ['sexo', 'parentesco', 'faixa_renda', 'escolaridade']

Note-se que a coluna de parentesco tem alguns poucos registros vazios. É possivel que sejam pessoas "sozinhas".

A escolaridade também possui registros vazios.

In [20]:
df[cols_categoricas].isnull().any()

sexo            False
parentesco       True
faixa_renda     False
escolaridade     True
dtype: bool

In [21]:
df['parentesco'].isnull().mean()

np.float64(0.0032214538368882605)

In [22]:
df['parentesco'].isnull().sum()

np.int64(1602)

In [23]:
df['escolaridade'].isnull().mean()

np.float64(0.08193592886257745)

In [24]:
df['escolaridade'].isnull().sum()

np.int64(40746)

Por conta disso essas colunas foram carregadas como float. Vou transformar em int.

In [25]:
df[cols_categoricas].dtypes

sexo              int64
parentesco      float64
faixa_renda       int64
escolaridade    float64
dtype: object

In [26]:
df['parentesco'] = df['parentesco'].apply(lambda x: int(x) if not pd.isnull(x) else 999)
df['escolaridade'] = df['escolaridade'].apply(lambda x: int(x) if not pd.isnull(x) else 999)

In [27]:
df[cols_categoricas].dtypes

sexo            int64
parentesco      int64
faixa_renda     int64
escolaridade    int64
dtype: object

In [28]:
mapper_cols_categoricas = {
    'sexo' : {
        1 : 'masculino',
        2 : 'feminino'
    },
    'faixa_renda' : {
        2 : 'R$ 0 a R$600,00',
        3 : 'R$601,00 a R%1.000,00',
        4 : 'R$1.001,00 ou mais'
    },
    'parentesco' : {
        1 : 'Pessoa Responsável Familiar',
        2 : 'Cônjuge/companheiro(a)',
        3 : 'Filho(a)',
        4 : 'Enteado(a)',
        5 : 'Neto(a) ou bisneto(a)',
        6 : 'Pai ou mãe',
        7 : 'Sogro(a)',
        8 : 'Irmão ou irmã',
        9 : 'Genro ou nora',
        10 : 'Outro parente',
        11 : 'Não parente'
    },
    'escolaridade' : {
        1 : 'Sem instrução',
        2 : 'Fundamental incompleto',
        3 : 'Fundamental completo',
        4 : 'Médio incompleto',
        5 : 'Médio completo',
        6 : 'Superior incompleto ou mais'
    }

}

In [29]:
for col, mapper in mapper_cols_categoricas.items():
    df[col] = df[col].map(mapper).fillna('Não  informado.')

In [30]:
for col in cols_categoricas:
    print(f'Contagem de valores: {col}')
    pprint(df[col].value_counts(normalize=True).to_dict(), indent=4)
    print('\n')

Contagem de valores: sexo
{'feminino': 0.5769056749468621, 'masculino': 0.4230943250531379}


Contagem de valores: parentesco
{   'Cônjuge/companheiro(a)': 0.10574894779917593,
    'Enteado(a)': 0.002652370543605253,
    'Filho(a)': 0.42494837027012355,
    'Genro ou nora': 0.0019364919131856398,
    'Irmão ou irmã': 0.010732146771206397,
    'Neto(a) ou bisneto(a)': 0.022226422758505584,
    'Não  informado.': 0.0032214538368882605,
    'Não parente': 0.002205951847107629,
    'Outro parente': 0.007211069574957118,
    'Pai ou mãe': 0.015365248918641198,
    'Pessoa Responsável Familiar': 0.4029833638654229,
    'Sogro(a)': 0.0007681619011805965}


Contagem de valores: faixa_renda
{   'R$ 0 a R$600,00': 0.47438220277463294,
    'R$1.001,00 ou mais': 0.2617360861145687,
    'R$601,00 a R%1.000,00': 0.2638817111107983}


Contagem de valores: escolaridade
{   'Fundamental completo': 0.07763060260491342,
    'Fundamental incompleto': 0.33760112288378435,
    'Médio completo': 0.1791546599

### Correção dos nomes dos bairros

Os dados geo contém uma tabela com a padronização dos nomes dos bairros. Vamos aplicar essa correção sobre a coluna localidade.

In [31]:
df['localidade'].isnull().any()

np.False_

In [32]:
def load_csv_geo(fname:str, **read_kwargs)->pd.DataFrame:
    """Carrega o arquivo CSV da base orginal e retorna um DataFrame."""

    file = os.path.join('data', 'geo', 'tabelas_geo', fname)
    if not os.path.exists(file):
        raise FileNotFoundError(f"Arquivo {file} não encontrado.")
    return pd.read_csv(file, **read_kwargs)

In [33]:
correcao = load_csv_geo('correcao_bairros.csv', sep=';')

In [34]:
correcao.head()

,Nomes errados ou incompletos,Código bairro oficial,Bairro oficial
0,Aeroporto,439,Aeroporto
1,CHACARA AEROPORTO,439,Aeroporto
2,Agapeama,415,Agapeama
3,AGAPEMA,415,Agapeama
4,AGAPRAMA,415,Agapeama


Note que as localidades estão todas com letras maiúsculas.

In [35]:
df['localidade'].str.isupper().all()

np.True_

In [36]:
correcao['nomes_originais_upper'] = correcao['Nomes errados ou incompletos'].str.upper()
correcao['bairro_oficial_upper'] = correcao['Bairro oficial'].str.upper()

In [37]:
mapper_bairros = dict(zip(correcao['nomes_originais_upper'], correcao['bairro_oficial_upper']))

Poucos bairros precisam ser corrigidos

In [38]:
df['localidade'].isin(mapper_bairros.keys()).value_counts(normalize=True)

localidade
True     0.999115
False    0.000885
Name: proportion, dtype: float64

In [39]:
df['localidade_padronizado'] = df['localidade'].apply(lambda x: mapper_bairros.get(x, x))

In [40]:
bairros_oficiais = load_csv_geo('bairros_oficiais.csv', sep=';')

In [41]:
bairros_oficiais.head()

,fid,cdbairro,cdregadm,nmbairro
0,1,418,S,CRISTAIS
1,2,404,N,FAZENDA CONCEIÇÃO
2,3,429,S,VILA RAMI
3,4,424,S,TIJUCO PRETO
4,5,417,S,CASTANHO


In [42]:
bairros_oficiais['nm_bairro_upper'] = bairros_oficiais['nmbairro'].str.upper()

In [43]:
nomes_oficiais = set(bairros_oficiais['nm_bairro_upper'].unique())

Note-se que mesmo com a padronização alguns bairros não foram encontrados.

In [44]:
df['localidade_padronizado'].isin(nomes_oficiais).value_counts(normalize=True)

localidade_padronizado
True     0.997414
False    0.002586
Name: proportion, dtype: float64

In [45]:
df['localidade_padronizado'].isin(nomes_oficiais).value_counts()

localidade_padronizado
True     496005
False      1286
Name: count, dtype: int64

Vou fazer o merge para obter as informações de código do bairro e de região administrativa

In [46]:
bairros_oficiais = bairros_oficiais[['nm_bairro_upper', 'cdregadm', 'cdbairro']].copy()

In [47]:
df = pd.merge(df, bairros_oficiais, how='left', left_on='localidade_padronizado', right_on='nm_bairro_upper')

In [48]:
df.head()

,cod_familiar,data_nascimento,sexo,parentesco,escolaridade,referencia_dados,localidade,faixa_renda,localidade_padronizado,nm_bairro_upper,cdregadm,cdbairro
0,X*HXV;+W,04/05/1967,feminino,Pessoa Responsável Familiar,Sem instrução,2024-01,VILA DELLA PIAZZA,"R$ 0 a R$600,00",VIANELO,VIANELO,S,425.0
1,X*HXV;+W,29/05/2006,feminino,Neto(a) ou bisneto(a),Fundamental incompleto,2024-01,VILA DELLA PIAZZA,"R$ 0 a R$600,00",VIANELO,VIANELO,S,425.0
2,X@;WXW?H,24/08/2006,feminino,Filho(a),Fundamental completo,2024-01,PARQUE ELOY CHAVES,"R$601,00 a R%1.000,00",ELOY CHAVES,ELOY CHAVES,W,446.0
3,X@;WXW?H,16/03/1984,feminino,Pessoa Responsável Familiar,Médio incompleto,2024-01,PARQUE ELOY CHAVES,"R$601,00 a R%1.000,00",ELOY CHAVES,ELOY CHAVES,W,446.0
4,**+@?*WXV,11/09/1992,feminino,Filho(a),Médio completo,2024-01,JARDIM SANTA GERTRUDES,"R$ 0 a R$600,00",SANTA GERTRUDES,SANTA GERTRUDES,S,422.0


In [49]:
df['nm_bairro_upper'].isnull().any()

np.True_

In [50]:
df['nm_bairro_upper'].fillna('Não identificado', inplace=True)

/tmp/ipykernel_250748/3930356949.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['nm_bairro_upper'].fillna('Não identificado', inplace=True)


In [51]:
dropar_cols_bairros = ['localidade', 'localidade_padronizado']

df.drop(columns=dropar_cols_bairros, inplace=True)
df.rename(columns={'nm_bairro_upper' : 'bairro_oficial'}, inplace=True)

In [52]:
df.head()

,cod_familiar,data_nascimento,sexo,parentesco,escolaridade,referencia_dados,faixa_renda,bairro_oficial,cdregadm,cdbairro
0,X*HXV;+W,04/05/1967,feminino,Pessoa Responsável Familiar,Sem instrução,2024-01,"R$ 0 a R$600,00",VIANELO,S,425.0
1,X*HXV;+W,29/05/2006,feminino,Neto(a) ou bisneto(a),Fundamental incompleto,2024-01,"R$ 0 a R$600,00",VIANELO,S,425.0
2,X@;WXW?H,24/08/2006,feminino,Filho(a),Fundamental completo,2024-01,"R$601,00 a R%1.000,00",ELOY CHAVES,W,446.0
3,X@;WXW?H,16/03/1984,feminino,Pessoa Responsável Familiar,Médio incompleto,2024-01,"R$601,00 a R%1.000,00",ELOY CHAVES,W,446.0
4,**+@?*WXV,11/09/1992,feminino,Filho(a),Médio completo,2024-01,"R$ 0 a R$600,00",SANTA GERTRUDES,S,422.0


Por fim, vamos pegar o nome da regiao admnistrativa que está no dicionario de dados geo.

In [53]:
dici_geo = load_csv_geo('dicionario_geo.csv', sep=';', skiprows=1)
dici_geo

,Nome,Descrição,Resposta
0,fid,Idenficador padrão,[código]
1,cdbairro,Código do bairro,[código]
2,cdregadm,Código da região,C = Central; W = Oeste; NW = Noroeste; N = Nor...
3,nmbairro,Nome do bairro,[nome do bairro oficial]


In [54]:
dici_geo.loc[2, 'Resposta']

'C = Central; W = Oeste; NW = Noroeste; N = Norte; NE = Nordeste; E = Leste; S = Sul; NA/Null = Serra do Japi'

In [55]:
df['cdregadm'].unique()

array(['S', 'W', 'E', 'N', 'NW', nan, 'NE', 'C'], dtype=object)

Temos que tomar cuidado para não colocar todos os bairros não identificados como Serra do Japi.

In [56]:
df[df['bairro_oficial']=='Não identificado']['cdregadm'].unique()

array([nan], dtype=object)

In [57]:
mapper_regiao_admin = {
    'C' : 'Central',
    'W' : 'Oeste',
    'NW' : 'Noroeste',
    'N' : 'Norte',
    'NE' : 'Nordeste',
    'E' : 'Leste',
    'S' : 'Sul',
    }



In [58]:
def fill_regiao_admin(row):

    if row['bairro_oficial']=='Não identificado':
        return 'Não identificado'
    
    return mapper_regiao_admin.get(row['cdregadm'], 'Serra do Japi')

In [59]:
df['regiao_administrativa'] = df.apply(fill_regiao_admin, axis=1)

In [60]:

df['regiao_administrativa'].value_counts(normalize=True)

regiao_administrativa
Oeste               0.271028
Leste               0.230147
Sul                 0.198403
Norte               0.134040
Noroeste            0.112890
Nordeste            0.030135
Central             0.016562
Serra do Japi       0.004209
Não identificado    0.002586
Name: proportion, dtype: float64

In [61]:
df.to_csv(os.path.join(DIR_DADOS_GERADOS, 'base_limpa.csv'), index=False)